In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf
from scipy.stats import pearsonr, spearmanr
from ta import add_all_ta_features
from ta.utils import dropna
from sklearn.preprocessing import MinMaxScaler

def analyze_technical_correlations(ticker):
    # Step 1: Download stock data
    print(f"📥 Downloading data for {ticker}...")
    end_date = datetime.today()
    start_date = end_date - relativedelta(days=60)

    df = yf.download(ticker, start=start_date, end=end_date, interval="1d", auto_adjust=True)
    if df.empty:
        print(f"❌ No data retrieved for {ticker}.")
        return

    df.reset_index(inplace=True)
    df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Step 2: Add all TA-lib indicators
    print("🧠 Computing all technical indicators...")
    df = dropna(df)
    df = add_all_ta_features(df,
                             open="Open", high="High", low="Low",
                             close="Close", volume="Volume",
                             fillna=True)

    df.reset_index(inplace=True)
    
    # Step 3: Normalize data
    print("📊 Normalizing numeric features...")
    df['Returns'] = df['Close'].pct_change()
    df['Lagged_Returns'] = df['Returns'].shift(1)
    df.dropna(inplace=True)

    scaler = MinMaxScaler()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Step 4: Select indicator columns (exclude target variables)
    exclude_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Returns', 'Lagged_Returns']
    indicator_cols = [col for col in df.columns if col not in exclude_cols]

    # Step 5: Correlation Analysis
    print("🔍 Calculating correlations...")
    pearson_results = {'Price': {}, 'Returns': {}, 'Lagged_Returns': {}}
    spearman_results = {'Price': {}, 'Returns': {}, 'Lagged_Returns': {}}

    def calculate_correlations(data, x_col, y_col, lag=0):
        x = data[x_col].shift(lag) if lag > 0 else data[x_col]
        y = data[y_col]
        valid_data = pd.concat([x, y], axis=1).replace([np.inf, -np.inf], np.nan).dropna()
        if len(valid_data) < 3:
            return np.nan, np.nan
        try:
            p_corr, _ = pearsonr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
            s_corr, _ = spearmanr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
            return p_corr, s_corr
        except:
            return np.nan, np.nan

    for col in indicator_cols:
        for label, y_col, lag in [('Price', 'Close', 0), ('Returns', 'Returns', 0), ('Lagged_Returns', 'Returns', 1)]:
            p_corr, s_corr = calculate_correlations(df, col, y_col, lag)
            pearson_results[label][col] = p_corr
            spearman_results[label][col] = s_corr

    pearson_df = pd.DataFrame(pearson_results)
    spearman_df = pd.DataFrame(spearman_results)

    # Step 6: Save correlation data
    # pearson_df.to_csv(f"{ticker}_pearson_correlations.csv")
    # spearman_df.to_csv(f"{ticker}_spearman_correlations.csv")

    # # Step 7: Visualize with heatmaps
    # print("📈 Creating heatmaps...")
    # plt.figure(figsize=(14, 10))
    # sns.heatmap(pearson_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    # plt.title(f"{ticker} - Pearson Correlation Heatmap")
    # plt.tight_layout()
    # plt.savefig(f"{ticker}_pearson_heatmap.png")
    # plt.show()

    # plt.figure(figsize=(14, 10))
    # sns.heatmap(spearman_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    # plt.title(f"{ticker} - Spearman Correlation Heatmap")
    # plt.tight_layout()
    # plt.savefig(f"{ticker}_spearman_heatmap.png")
    # plt.show()

    # Step 8: Show strong correlations
    print(f"\n📌 Significant Pearson Correlations (|corr| > 0.85 for {ticker}):")
    # sort  by absolute value and filter
    pearson_df = pearson_df.abs().sort_values(by='Price', ascending=False)
    print(pearson_df[(pearson_df['Price'].abs() > 0.85)])
    # store them in a file 
    pearson_df[(pearson_df['Price'].abs() > 0.7)].to_csv(f"Pearsons_Coeff_Sixty_Day/{ticker}_significant_pearson.csv")

    # print("\n📌 Significant Spearman Correlations (|corr| > 0.7):")
    # print(spearman_df[(spearman_df['Returns'].abs() > 0.7)])



    # def plot_significant_indicators(df, indicator_list):
    #     print("\n📉 Plotting significant indicators with Close price...")
    #     df_plot = df.set_index('Date')
    #     for indicator in indicator_list:
    #         if indicator not in df_plot.columns:
    #             continue
    #         plt.figure(figsize=(14, 6))
    #         plt.plot(df_plot['Close'], label='Close Price', linewidth=2)
    #         plt.plot(df_plot[indicator], label=indicator, linestyle='--')
    #         plt.title(f"{ticker} - Close vs {indicator}")
    #         plt.xlabel("Date")
    #         plt.ylabel("Normalized Value")
    #         plt.legend()
    #         plt.tight_layout()
    #         # plt.savefig(f"{ticker}_Close_vs_{indicator}.png")
    #         plt.show()

    # significant_indicators = pearson_df.index[pearson_df['Price'].abs() > 0.5].tolist()
    # plot_significant_indicators(df, significant_indicators)


In [16]:
analyze_technical_correlations("TCS.NS")  #Large Cap IT
analyze_technical_correlations("INFY.NS") #large Cap IT
analyze_technical_correlations("WIPRO.NS")  #Large Cap IT
analyze_technical_correlations("HCLTECH.NS") # Large Cap IT

analyze_technical_correlations("TECHM.NS")  # Mid Cap IT
analyze_technical_correlations("PERSISTENT.NS")  # Mid Cap IT
# Small Cap IT
analyze_technical_correlations("LTIM.NS")  # Small Cap IT


📥 Downloading data for TCS.NS...


[*********************100%***********************]  1 of 1 completed
/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:65: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  p_corr, _ = pearsonr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:66: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s_corr, _ = spearmanr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:65: Con

🧠 Computing all technical indicators...
📊 Normalizing numeric features...
🔍 Calculating correlations...

📌 Significant Pearson Correlations (|corr| > 0.85 for TCS.NS):
           Price   Returns  Lagged_Returns
others_cr    1.0  0.314618        0.445818
📥 Downloading data for INFY.NS...


[*********************100%***********************]  1 of 1 completed
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:65: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  p_corr, _ = pearsonr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:66: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s_corr, _ = spearmanr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:65: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  p_corr, _ = pearsonr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:66: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s_corr, _ = spear

🧠 Computing all technical indicators...
📊 Normalizing numeric features...
🔍 Calculating correlations...

📌 Significant Pearson Correlations (|corr| > 0.85 for INFY.NS):
                         Price   Returns  Lagged_Returns
others_cr             1.000000  0.106154        0.265887
volume_vpt            0.982620  0.128443        0.315356
trend_ichimoku_a      0.943248  0.084235        0.142492
momentum_kama         0.941602  0.113722        0.150813
trend_ichimoku_base   0.939199  0.064823        0.148607
trend_ichimoku_conv   0.926379  0.098884        0.134181
trend_ichimoku_b      0.922698  0.076055        0.215450
volatility_dch        0.921437  0.077471        0.221149
trend_ema_fast        0.917679  0.147126        0.114175
volatility_dcm        0.914161  0.072684        0.127465
trend_sma_slow        0.895139  0.125380        0.063251
trend_ema_slow        0.892614  0.134018        0.084970
momentum_tsi          0.892023  0.152405        0.099302
volatility_kcc        0.884470  0

...

🧠 Computing all technical indicators...
📊 Normalizing numeric features...
🔍 Calculating correlations...

📌 Significant Pearson Correlations (|corr| > 0.85 for TECHM.NS):
                        Price   Returns  Lagged_Returns
others_cr            1.000000  0.032063        0.202749
volume_vpt           0.983075  0.116507        0.204604
volatility_dch       0.943220  0.210390        0.194886
trend_ichimoku_b     0.943220  0.210390        0.194886
momentum_kama        0.919459  0.248157        0.189584
trend_ichimoku_a     0.914853  0.160228        0.104714
trend_aroon_up       0.914388  0.126028        0.175807
trend_ema_fast       0.906055  0.253886        0.170749
trend_ichimoku_conv  0.902093  0.197724        0.132945
volatility_bbh       0.887800  0.319925        0.245374
trend_ema_slow       0.873511  0.214949        0.122294
trend_sma_slow       0.866803  0.194952        0.082951
volatility_kch       0.866527  0.287640        0.190386
trend_ichimoku_base  0.861550  0.107508       

[*********************100%***********************]  1 of 1 completed
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:65: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  p_corr, _ = pearsonr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:66: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s_corr, _ = spearmanr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:65: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  p_corr, _ = pearsonr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:66: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s_corr, _ = spear

🧠 Computing all technical indicators...
📊 Normalizing numeric features...
🔍 Calculating correlations...

📌 Significant Pearson Correlations (|corr| > 0.85 for PERSISTENT.NS):
                         Price   Returns  Lagged_Returns
others_cr             1.000000  0.032912        0.312909
volume_vpt            0.966627  0.045444        0.327457
trend_aroon_up        0.894823  0.183553        0.294988
trend_ichimoku_a      0.886992  0.182780        0.179349
momentum_kama         0.884743  0.232364        0.198320
trend_vortex_ind_pos  0.883377  0.111571        0.335713
trend_ema_fast        0.881450  0.242798        0.197663
trend_ichimoku_b      0.876723  0.205140        0.239940
volatility_dch        0.876723  0.205140        0.239940
trend_ichimoku_conv   0.873516  0.222325        0.212770
volatility_bbh        0.871044  0.273544        0.244172
📥 Downloading data for LTIM.NS...


[*********************100%***********************]  1 of 1 completed
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:65: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  p_corr, _ = pearsonr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:66: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s_corr, _ = spearmanr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:65: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  p_corr, _ = pearsonr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
/var/folders/1v/3qv219b14cqbcl96rxmr2pg00000gq/T/ipykernel_6483/3738087352.py:66: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  s_corr, _ = spear

🧠 Computing all technical indicators...
📊 Normalizing numeric features...
🔍 Calculating correlations...

📌 Significant Pearson Correlations (|corr| > 0.85 for LTIM.NS):
                        Price   Returns  Lagged_Returns
others_cr            1.000000  0.059484        0.155165
trend_ichimoku_b     0.986608  0.140563        0.138746
volatility_dch       0.986608  0.140563        0.138746
trend_ema_fast       0.975119  0.190228        0.110942
momentum_kama        0.974131  0.203608        0.138642
volume_vpt           0.974010  0.033079        0.156677
trend_ichimoku_conv  0.967855  0.144275        0.098227
trend_ichimoku_a     0.967216  0.155136        0.126662
volatility_kch       0.966552  0.198045        0.103138
trend_ema_slow       0.964886  0.183890        0.103571
volatility_kcc       0.963271  0.198398        0.100750
volatility_dcm       0.962586  0.140964        0.116708
volatility_bbh       0.962560  0.209220        0.158944
trend_sma_fast       0.962215  0.191684        

SENSITIVITY ANALYSIS

In [11]:
def sensitivity_analysis(ticker, period=1):
    import pandas as pd
    import numpy as np
    import yfinance as yf
    import matplotlib.pyplot as plt
    from ta import add_all_ta_features
    from ta.utils import dropna
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    from scipy.stats import zscore

    print(f"📥 Downloading data for {ticker}...")
    end_date = datetime.today()
    start_date = end_date - relativedelta(days=90)

    df = yf.download(ticker, start=start_date, end=end_date, interval="1d", auto_adjust=True)
    if df.empty:
        print(f"❌ No data retrieved for {ticker}.")
        return

    df.reset_index(inplace=True)
    df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Step 2: Add technical indicators
    print("🧠 Computing all technical indicators...")
    df = dropna(df)
    df = add_all_ta_features(df,
                             open="Open", high="High", low="Low",
                             close="Close", volume="Volume",
                             fillna=True)
    df.reset_index(inplace=True)

    # Z-score normalization for numeric columns
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = df[numeric_cols].apply(zscore)
    df.dropna(inplace=True)

    # Calculate price change
    df['Price_Change'] = df['Close'].diff(periods=period)
    df.dropna(inplace=True)

    # Identify indicator columns (exclude OHLCV and computed returns)
    exclude_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Price_Change']
    indicator_cols = [col for col in df.columns if col not in exclude_cols]

    # Compute sensitivity scores
    sensitivity_scores = {}

    for col in indicator_cols:
        if df[col].isnull().any():
            continue
        df[f'{col}_Change'] = df[col].diff(periods=period)
        df_filtered = df[[f'{col}_Change', 'Price_Change']].replace([np.inf, -np.inf], np.nan).dropna()

        # Avoid divide-by-zero
        df_filtered = df_filtered[df_filtered[f'{col}_Change'] != 0]

        if df_filtered.empty:
            sensitivity_scores[col] = np.nan
            continue

        # Clip extreme values to prevent distortion
        sensitivities = (df_filtered['Price_Change'] / df_filtered[f'{col}_Change']).clip(lower=-10, upper=10)
        sensitivity_scores[col] = sensitivities.mean()

    # Construct sensitivity DataFrame
    sensitivity_df = pd.DataFrame.from_dict(sensitivity_scores, orient='index', columns=['Average_Sensitivity'])
    sensitivity_df.dropna(inplace=True)

    # Compute Z-score of sensitivities
    sensitivity_df['Zscore_Sensitivity'] = zscore(sensitivity_df['Average_Sensitivity'])

    # Sort for visualization
    sensitivity_df.sort_values(by='Average_Sensitivity', ascending=False, inplace=True)
    top_raw = sensitivity_df.head(20)

    sensitivity_df.sort_values(by='Zscore_Sensitivity', ascending=False, inplace=True)
    top_zscore = sensitivity_df.head(20)

    # --- Plot Raw + Z-score Side-by-Side ---
    # fig, axes = plt.subplots(1, 2, figsize=(18, 6))

    # Plot 1: Raw Sensitivity Scores
    # top_raw['Average_Sensitivity'].plot(kind='bar', ax=axes[0])
    # axes[0].set_title(f"{ticker} - Top 20 Raw Sensitivity Scores")
    # axes[0].set_ylabel("Average ΔPrice / ΔIndicator")
    # axes[0].set_xlabel("Technical Indicator")
    # axes[0].tick_params(axis='x', rotation=90)
    # axes[0].grid(True)

    # Plot 2: Z-score Normalized Sensitivity
    # top_zscore['Zscore_Sensitivity'].plot(kind='bar', ax=axes[1], color='orange')
    # axes[1].set_title(f"{ticker} - Top 20 Normalized (Z-score) Sensitivity")
    # axes[1].set_ylabel("Z-score of Sensitivity")
    # axes[1].set_xlabel("Technical Indicator")
    # axes[1].tick_params(axis='x', rotation=90)
    # axes[1].grid(True)

    # plt.tight_layout()
    # plt.savefig(f"{ticker}_sensitivity_comparison.png")
    # plt.show()

    # 📋 Print top 20 indicators
    print("\n📌 Top 20 Indicators by Raw Average Sensitivity:")
    print(top_raw[['Average_Sensitivity']])

    print("\n📌 Top 20 Indicators by Z-score Normalized Sensitivity:")
    print(top_zscore[['Zscore_Sensitivity']])
    # store top 20 Z-score indicators in a file
    top_zscore[['Zscore_Sensitivity']].to_csv(f"Sensitivity_60/{ticker}_top_sensitivity_indicators.csv")

    # Optional: Save to CSV
    # sensitivity_df.to_csv(f"{ticker}_sensitivity_scores.csv")

    return sensitivity_df


In [17]:
sensitivity_analysis("TCS.NS", period=1)
sensitivity_analysis("INFY.NS", period=1)
sensitivity_analysis("WIPRO.NS", period=1)
sensitivity_analysis("HCLTECH.NS", period=1)
sensitivity_analysis("TECHM.NS", period=1)
sensitivity_analysis("PERSISTENT.NS", period=1)
sensitivity_analysis("LTIM.NS", period=1)

📥 Downloading data for TCS.NS...


[*********************100%***********************]  1 of 1 completed


🧠 Computing all technical indicators...

📌 Top 20 Indicators by Raw Average Sensitivity:
                       Average_Sensitivity
momentum_tsi                      2.294163
volume_vpt                        1.854567
trend_kst                         1.572366
momentum_rsi                      1.537790
trend_aroon_ind                   1.451901
momentum_ao                       1.446071
trend_ema_fast                    1.443943
momentum_kama                     1.409822
volume_nvi                        1.394672
volume_sma_em                     1.215497
momentum_stoch_rsi_k              1.203431
trend_adx_pos                     1.149208
trend_ichimoku_a                  1.144070
trend_vortex_ind_diff             1.099720
trend_cci                         1.066570
trend_kst_diff                    1.053283
trend_sma_slow                    1.029747
others_cr                         1.000000
volatility_dcp                    0.962440
momentum_stoch_rsi                0.937596

📌 Top 2

[*********************100%***********************]  1 of 1 completed
/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


🧠 Computing all technical indicators...

📌 Top 20 Indicators by Raw Average Sensitivity:
                      Average_Sensitivity
trend_ichimoku_b                 1.984622
volatility_dcl                   1.893499
trend_aroon_ind                  1.745005
volume_vpt                       1.617993
trend_adx                        1.580439
momentum_kama                    1.511227
volatility_dcm                   1.325164
momentum_ppo_signal              1.240192
trend_macd_signal                1.225186
trend_macd                       1.212264
trend_sma_fast                   1.200443
momentum_rsi                     1.179562
trend_adx_pos                    1.177760
volatility_bbp                   1.145156
momentum_roc                     1.036442
volume_obv                       1.017217
momentum_ppo_hist                1.009957
trend_vortex_ind_pos             1.000724
others_cr                        1.000000
momentum_stoch_rsi               0.981854

📌 Top 20 Indicators by Z-sco

...

🧠 Computing all technical indicators...

📌 Top 20 Indicators by Raw Average Sensitivity:
                         Average_Sensitivity
volume_vpt                          3.053193
volume_fi                           1.604214
volume_sma_em                       1.408940
trend_adx_pos                       1.260871
volatility_dcw                      1.229263
volatility_bbl                      1.081056
volatility_kcc                      1.042597
others_cr                           1.000000
momentum_rsi                        0.989321
momentum_stoch_rsi                  0.974491
trend_adx                           0.969010
momentum_stoch_signal               0.962124
volatility_dcm                      0.937549
volatility_dcl                      0.925343
momentum_ppo_hist                   0.924217
trend_macd_diff                     0.885198
trend_kst_diff                      0.846530
volatility_kcl                      0.807283
trend_visual_ichimoku_b             0.774198
trend_ema_f

[*********************100%***********************]  1 of 1 completed

🧠 Computing all technical indicators...

📌 Top 20 Indicators by Raw Average Sensitivity:
                     Average_Sensitivity
volume_vpt                      2.053619
volatility_dch                  1.865482
trend_ichimoku_b                1.744374
volatility_dcm                  1.552207
momentum_tsi                    1.335869
trend_ichimoku_conv             1.244373
trend_macd                      1.113297
momentum_kama                   1.067625
trend_ichimoku_base             1.063925
trend_ichimoku_a                1.058966
others_cr                       1.000000
volatility_kch                  0.857071
volatility_bbp                  0.851466
trend_psar_up                   0.850664
trend_cci                       0.813804
trend_aroon_ind                 0.803121
momentum_rsi                    0.800719
momentum_ao                     0.784000
trend_kst                       0.774330
volatility_kcc                  0.760839

📌 Top 20 Indicators by Z-score Normalized Sensiti


/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


,Average_Sensitivity,Zscore_Sensitivity
volume_vpt,2.053619,2.444279
volatility_dch,1.865482,2.174751
trend_ichimoku_b,1.744374,2.001249
volatility_dcm,1.552207,1.725947
momentum_tsi,1.335869,1.416018
...,...,...
volatility_kcw,-0.800445,-1.644509
volatility_ui,-0.848704,-1.713646
volatility_bbh,-0.887285,-1.768917
trend_psar_down,-1.174796,-2.180811


In [7]:
def combined_indicator_analysis(ticker, period=1):
    import pandas as pd
    import numpy as np
    import yfinance as yf
    import matplotlib.pyplot as plt
    from ta import add_all_ta_features
    from ta.utils import dropna
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    from scipy.stats import pearsonr, zscore

    print(f"📥 Downloading data for {ticker}...")
    end_date = datetime.today()
    start_date = end_date - relativedelta(days=90)

    df = yf.download(ticker, start=start_date, end=end_date, interval="1d", auto_adjust=True)
    if df.empty:
        print(f"❌ No data retrieved for {ticker}.")
        return None

    df.reset_index(inplace=True)
    df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Compute technical indicators
    print("🧠 Computing all technical indicators...")
    df = dropna(df)
    df = add_all_ta_features(df,
                             open="Open", high="High", low="Low",
                             close="Close", volume="Volume",
                             fillna=True)
    df.reset_index(inplace=True)

    # Normalize numeric features using Z-score
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = df[numeric_cols].apply(lambda x: zscore(x) if x.std() != 0 else np.nan)
    df.dropna(inplace=True)

    # Calculate price change
    df['Price_Change'] = df['Close'].diff(periods=period)
    df.dropna(inplace=True)

    # Identify indicator columns
    exclude_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Price_Change']
    indicator_cols = [col for col in df.columns if col not in exclude_cols]

    # Initialize dictionaries
    correlation_scores = {}
    sensitivity_scores = {}

    for col in indicator_cols:
        if df[col].isnull().any():
            continue

        # --- Correlation Calculation ---
        try:
            valid_corr = df[[col, 'Close']].replace([np.inf, -np.inf], np.nan).dropna()
            if len(valid_corr) >= 3:
                corr, _ = pearsonr(valid_corr[col], valid_corr['Close'])
                correlation_scores[col] = corr
            else:
                correlation_scores[col] = np.nan
        except:
            correlation_scores[col] = np.nan

        # --- Sensitivity Calculation ---
        df[f'{col}_Change'] = df[col].diff(periods=period)
        df_filtered = df[[f'{col}_Change', 'Price_Change']].replace([np.inf, -np.inf], np.nan).dropna()
        df_filtered = df_filtered[df_filtered[f'{col}_Change'] != 0]

        if df_filtered.shape[0] < 5:
            sensitivity_scores[col] = np.nan
            continue

        sensitivities = (df_filtered['Price_Change'] / df_filtered[f'{col}_Change']).clip(lower=-10, upper=10)
        sensitivity_scores[col] = sensitivities.mean()

    # Create DataFrame
    combined_df = pd.DataFrame({
        'Pearson_Correlation': correlation_scores,
        'Average_Sensitivity': sensitivity_scores
    })

    combined_df.dropna(inplace=True)

    # Z-score normalize sensitivity
    combined_df['Zscore_Sensitivity'] = zscore(combined_df['Average_Sensitivity'])

    # Compute combined score: Correlation × Z-score Sensitivity
    combined_df['Combined_Score'] = combined_df['Pearson_Correlation'] * combined_df['Zscore_Sensitivity']

    # Sort and return
    combined_df.sort_values(by='Combined_Score', ascending=False, inplace=True)

    print("\n📊 Top Indicators by Combined Score:")
    print(combined_df[['Pearson_Correlation', 'Zscore_Sensitivity', 'Combined_Score']].head(15))

    return combined_df


In [8]:
combined_indicator_analysis("TCS.NS", period=1)
combined_indicator_analysis("INFY.NS", period=1)
combined_indicator_analysis("WIPRO.NS", period=1)
combined_indicator_analysis("HCLTECH.NS", period=1)
combined_indicator_analysis("TECHM.NS", period=1)
combined_indicator_analysis("PERSISTENT.NS", period=1)
combined_indicator_analysis("LTIM.NS", period=1)

📥 Downloading data for TCS.NS...


[*********************100%***********************]  1 of 1 completed
/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


🧠 Computing all technical indicators...

📊 Top Indicators by Combined Score:
                      Pearson_Correlation  Zscore_Sensitivity  Combined_Score
volume_vpt                       0.849663            1.956162        1.662078
momentum_rsi                     0.794205            1.798401        1.428298
momentum_pvo_signal             -0.687514           -1.988123        1.366862
trend_dpo                       -0.556563           -2.231984        1.242239
momentum_kama                    0.550656            1.875319        1.032656
trend_vortex_ind_neg            -0.671772           -1.474859        0.990768
momentum_tsi                     0.449503            1.885939        0.847735
volume_sma_em                    0.824372            1.025912        0.845733
trend_ema_fast                   0.652503            1.281851        0.836412
momentum_pvo                    -0.649546           -1.259192        0.817903
others_cr                        1.000000            0.773027    

[*********************100%***********************]  1 of 1 completed
/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


🧠 Computing all technical indicators...

📊 Top Indicators by Combined Score:
                      Pearson_Correlation  Zscore_Sensitivity  Combined_Score
volume_vpt                       0.972075            1.958750        1.904052
volatility_dcl                   0.792513            1.819671        1.442112
trend_adx                        0.747101            1.851584        1.383319
momentum_kama                    0.856748            1.400484        1.199861
momentum_pvo_signal             -0.713985           -1.679872        1.199403
trend_aroon_down                -0.763449           -1.566309        1.195796
trend_macd                       0.921285            1.212534        1.117090
trend_vortex_ind_neg            -0.485817           -2.256757        1.096370
trend_aroon_ind                  0.800999            1.289843        1.033163
others_cr                        1.000000            0.936518        0.936518
momentum_rsi                     0.827741            1.111857    

...

[*********************100%***********************]  1 of 1 completed
/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


🧠 Computing all technical indicators...

📊 Top Indicators by Combined Score:
                      Pearson_Correlation  Zscore_Sensitivity  Combined_Score
volume_vpt                       0.711406            3.784940        2.692630
trend_adx                        0.775507            1.897347        1.471405
momentum_pvo_signal             -0.807573           -1.515606        1.223962
volatility_kcw                  -0.863225           -1.326225        1.144831
volume_fi                        0.757548            1.456355        1.103259
volatility_dcl                   0.672458            1.385178        0.931474
others_cr                        1.000000            0.908788        0.908788
trend_kst_diff                   0.527236            1.714373        0.903880
volatility_dcm                   0.715373            1.208016        0.864182
volatility_bbl                   0.674214            1.202667        0.810855
trend_dpo                       -0.517138           -1.526111    

[*********************100%***********************]  1 of 1 completed
/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


🧠 Computing all technical indicators...

📊 Top Indicators by Combined Score:
                     Pearson_Correlation  Zscore_Sensitivity  Combined_Score
volume_vpt                      0.973642            2.482898        2.417453
volatility_dch                  0.948588            2.343437        2.222955
volatility_dcm                  0.915514            2.000522        1.831505
trend_ichimoku_b                0.802179            2.235232        1.793056
trend_ichimoku_conv             0.958075            1.565119        1.499501
momentum_pvo_signal            -0.813510           -1.774861        1.443868
trend_macd                      0.956027            1.451313        1.387495
momentum_tsi                    0.867741            1.511340        1.311452
volatility_ui                  -0.457129           -2.720799        1.243756
volatility_kcw                 -0.806585           -1.473114        1.188192
trend_ichimoku_a                0.933649            1.245124        1.162509

,Pearson_Correlation,Average_Sensitivity,Zscore_Sensitivity,Combined_Score
volume_vpt,0.973642,2.097993,2.482898,2.417453
volatility_dch,0.948588,2.004714,2.343437,2.222955
volatility_dcm,0.915514,1.775355,2.000522,1.831505
trend_ichimoku_b,0.802179,1.932341,2.235232,1.793056
trend_ichimoku_conv,0.958075,1.484134,1.565119,1.499501
...,...,...,...,...
volatility_bbm,0.896807,0.114508,-0.482606,-0.432805
volatility_bbl,0.798453,0.060620,-0.563174,-0.449668
trend_sma_slow,0.864958,0.007842,-0.642082,-0.555374
volatility_bbh,0.896764,-0.586629,-1.530874,-1.372833


In [22]:
def combined_correlation_sensitivity_analysis(ticker, period=1):
    import pandas as pd
    import numpy as np
    import yfinance as yf
    from ta import add_all_ta_features
    from ta.utils import dropna
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    from sklearn.preprocessing import MinMaxScaler
    from scipy.stats import pearsonr, zscore

    print(f"📥 Downloading data for {ticker}...")
    end_date = datetime.today()
    start_date = end_date - relativedelta(days=90)

    df = yf.download(ticker, start=start_date, end=end_date, interval="1d", auto_adjust=True)
    if df.empty:
        print(f"❌ No data retrieved for {ticker}.")
        return

    df.reset_index(inplace=True)
    df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Step 1: Add technical indicators
    print("🧠 Computing all technical indicators...")
    df = dropna(df)
    df = add_all_ta_features(df, open="Open", high="High", low="Low",
                             close="Close", volume="Volume", fillna=True)
    df.reset_index(inplace=True)

    # ----- CORRELATION SECTION -----
    df_corr = df.copy()
    df_corr['Returns'] = df_corr['Close'].pct_change()
    df_corr['Lagged_Returns'] = df_corr['Returns'].shift(1)
    df_corr.dropna(inplace=True)

    scaler = MinMaxScaler()
    numeric_cols_corr = df_corr.select_dtypes(include=['float64', 'int64']).columns
    df_corr[numeric_cols_corr] = scaler.fit_transform(df_corr[numeric_cols_corr])

    exclude_cols_corr = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Returns', 'Lagged_Returns']
    indicator_cols_corr = [col for col in df_corr.columns if col not in exclude_cols_corr]

    pearson_scores = {}

    def calc_pearson(x_col, y_col):
        valid = pd.concat([x_col, y_col], axis=1).replace([np.inf, -np.inf], np.nan).dropna()
        if len(valid) < 3:
            return np.nan
        try:
            corr, _ = pearsonr(valid.iloc[:, 0], valid.iloc[:, 1])
            return corr
        except:
            return np.nan

    for col in indicator_cols_corr:
        pearson_scores[col] = calc_pearson(df_corr[col], df_corr['Close'])

    pearson_df = pd.DataFrame.from_dict(pearson_scores, orient='index', columns=['Pearson_Correlation'])
    #print top 20 indicators by absolute Pearson correlation
    pearson_df = pearson_df.abs().sort_values(by='Pearson_Correlation', ascending=False)
    print("\n📌 Top 20 Indicators by Absolute Pearson Correlation:" )
    print(pearson_df.head(20))
    pearson_df.dropna(inplace=True)

    # ----- SENSITIVITY SECTION -----
    df_sens = df.copy()
    numeric_cols_sens = df_sens.select_dtypes(include=['float64', 'int64']).columns
    df_sens[numeric_cols_sens] = df_sens[numeric_cols_sens].apply(lambda x: zscore(x) if x.std() != 0 else np.nan)
    df_sens.dropna(inplace=True)

    df_sens['Price_Change'] = df_sens['Close'].diff(periods=period)
    df_sens.dropna(inplace=True)

    exclude_cols_sens = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Price_Change']
    indicator_cols_sens = [col for col in df_sens.columns if col not in exclude_cols_sens]

    sensitivity_scores = {}

    for col in indicator_cols_sens:
        if df_sens[col].isnull().any():
            continue
        df_sens[f'{col}_Change'] = df_sens[col].diff(periods=period)
        df_filtered = df_sens[[f'{col}_Change', 'Price_Change']].replace([np.inf, -np.inf], np.nan).dropna()
        df_filtered = df_filtered[df_filtered[f'{col}_Change'] != 0]

        if df_filtered.shape[0] < 5:
            sensitivity_scores[col] = np.nan
            continue

        sensitivities = (df_filtered['Price_Change'] / df_filtered[f'{col}_Change']).clip(lower=-10, upper=10)
        sensitivity_scores[col] = sensitivities.mean()

    sensitivity_df = pd.DataFrame.from_dict(sensitivity_scores, orient='index', columns=['Average_Sensitivity'])
    sensitivity_df.dropna(inplace=True)
    sensitivity_df['Zscore_Sensitivity'] = zscore(sensitivity_df['Average_Sensitivity'])
    #print top 20 indicators by Z-score sensitivity
    sensitivity_df.sort_values(by='Zscore_Sensitivity', ascending=False, inplace=True)
    print("\n📌 Top 20 Indicators by Z-score Normalized Sensitivity:"   )
    print(sensitivity_df.head(20))

    # ----- COMBINE SECTION -----
    combined_df = pearson_df.join(sensitivity_df[['Zscore_Sensitivity']], how='inner')
    combined_df['Combined_Score'] = combined_df['Pearson_Correlation'] * combined_df['Zscore_Sensitivity']
    combined_df.sort_values(by='Combined_Score', ascending=False, inplace=True)

    # Save to CSV
    filename = f"Combined/{ticker}_combined_indicator_scores.csv"
    combined_df.to_csv(filename)
    print(f"✅ Combined score saved to {filename}")

    # Show top indicators
    print("\n📊 Top Indicators by Combined Score:")
    print(combined_df[['Pearson_Correlation', 'Zscore_Sensitivity', 'Combined_Score']].head(15))

    return combined_df


In [23]:
combined_correlation_sensitivity_analysis("TCS.NS", period=1)

📥 Downloading data for TCS.NS...


[*********************100%***********************]  1 of 1 completed
/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


🧠 Computing all technical indicators...

📌 Top 20 Indicators by Absolute Pearson Correlation:
                       Pearson_Correlation
others_cr                         1.000000
volume_fi                         0.848073
trend_macd                        0.835484
volume_vpt                        0.833717
momentum_ppo                      0.832187
volume_sma_em                     0.810525
momentum_rsi                      0.781348
momentum_ao                       0.778319
trend_ichimoku_conv               0.745427
volatility_bbw                    0.744942
trend_kst_diff                    0.725346
momentum_pvo_signal               0.709067
trend_vortex_ind_diff             0.703428
momentum_ppo_hist                 0.688113
trend_macd_diff                   0.684983
momentum_pvo                      0.684466
volume_adi                        0.657678
trend_vortex_ind_neg              0.652743
volatility_dcw                    0.649727
trend_ichimoku_a                  0.636203

📌 

,Pearson_Correlation,Zscore_Sensitivity,Combined_Score
volume_vpt,0.833717,1.961250,1.635127
momentum_rsi,0.781348,1.788044,1.397084
momentum_ao,0.778319,1.095776,0.852863
volume_sma_em,0.810525,1.023400,0.829491
momentum_kama,0.495761,1.615773,0.801037
...,...,...,...
volatility_bbl,0.617092,-1.265540,-0.780954
momentum_pvo,0.684466,-1.287539,-0.881277
trend_vortex_ind_neg,0.652743,-1.466119,-0.956999
trend_dpo,0.529204,-2.234835,-1.182683


In [24]:
combined_correlation_sensitivity_analysis("INFY.NS", period=1)
combined_correlation_sensitivity_analysis("WIPRO.NS", period=1)
combined_correlation_sensitivity_analysis("HCLTECH.NS", period=1)
combined_correlation_sensitivity_analysis("TECHM.NS", period=1)
combined_correlation_sensitivity_analysis("PERSISTENT.NS", period=1)
combined_correlation_sensitivity_analysis("LTIM.NS", period=1)

📥 Downloading data for INFY.NS...


[*********************100%***********************]  1 of 1 completed
/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


🧠 Computing all technical indicators...

📌 Top 20 Indicators by Absolute Pearson Correlation:
                     Pearson_Correlation
others_cr                       1.000000
volume_vpt                      0.971633
trend_macd                      0.930498
momentum_ppo                    0.926825
trend_ichimoku_conv             0.881980
volume_obv                      0.863332
momentum_kama                   0.861530
momentum_ao                     0.857634
momentum_rsi                    0.853604
trend_ichimoku_a                0.836977
trend_macd_signal               0.836224
momentum_ppo_signal             0.831139
trend_aroon_ind                 0.823149
trend_ema_fast                  0.822540
trend_trix                      0.806811
trend_kst                       0.798361
trend_adx                       0.795110
volatility_dcl                  0.793596
volatility_bbw                  0.791530
volatility_kcl                  0.788781

📌 Top 20 Indicators by Z-score Normalized Se

[*********************100%***********************]  1 of 1 completed
/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


🧠 Computing all technical indicators...

📌 Top 20 Indicators by Absolute Pearson Correlation:
                      Pearson_Correlation
others_cr                        1.000000
volume_vpt                       0.849301
volatility_dcl                   0.837281
volume_obv                       0.827045
trend_ichimoku_conv              0.809167
momentum_ppo                     0.792548
trend_macd                       0.790959
momentum_tsi                     0.755367
trend_macd_signal                0.746698
volume_mfi                       0.744636
volatility_dcw                   0.743883
trend_vortex_ind_neg             0.743880
momentum_ppo_signal              0.742759
volume_fi                        0.720014
trend_trix                       0.719787
volatility_bbl                   0.718547
trend_ichimoku_a                 0.711063
volatility_kcl                   0.705715
trend_kst                        0.705192
trend_ema_fast                   0.701151

📌 Top 20 Indicators by 

...

[*********************100%***********************]  1 of 1 completed
/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


🧠 Computing all technical indicators...

📌 Top 20 Indicators by Absolute Pearson Correlation:
                     Pearson_Correlation
others_cr                       1.000000
trend_macd                      0.912459
momentum_ppo                    0.905003
momentum_kama                   0.888898
volatility_kcw                  0.866886
momentum_rsi                    0.859060
trend_ichimoku_conv             0.857663
trend_ema_fast                  0.857582
momentum_ao                     0.818134
trend_ichimoku_a                0.815400
volume_vwap                     0.810912
volatility_kcl                  0.807581
momentum_pvo_signal             0.805790
trend_adx                       0.790278
trend_macd_signal               0.781258
volatility_kcc                  0.778162
momentum_ppo_signal             0.774201
volume_fi                       0.774055
trend_kst                       0.773473
momentum_pvo                    0.773342

📌 Top 20 Indicators by Z-score Normalized Se

[*********************100%***********************]  1 of 1 completed
/Users/utsavdhanuka/miniconda3/envs/env_pytorch/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


🧠 Computing all technical indicators...

📌 Top 20 Indicators by Absolute Pearson Correlation:
                     Pearson_Correlation
others_cr                       1.000000
volume_vpt                      0.973432
momentum_kama                   0.964620
trend_ichimoku_conv             0.958896
trend_macd                      0.955493
trend_ema_fast                  0.952842
volatility_dch                  0.950652
trend_adx                       0.945709
trend_kst                       0.942743
momentum_ppo                    0.942255
trend_macd_signal               0.940138
volatility_kch                  0.939309
volatility_kcc                  0.939225
volatility_kcl                  0.938120
trend_ichimoku_a                0.935687
momentum_ppo_signal             0.933004
trend_sma_fast                  0.931003
trend_trix                      0.927320
volume_vwap                     0.925260
volume_nvi                      0.923792

📌 Top 20 Indicators by Z-score Normalized Se

,Pearson_Correlation,Zscore_Sensitivity,Combined_Score
volume_vpt,0.973432,2.409339,2.345327
volatility_dch,0.950652,2.338172,2.222787
volatility_dcm,0.917862,2.195095,2.014793
trend_ichimoku_b,0.819093,2.229300,1.826003
trend_ichimoku_conv,0.958896,1.650474,1.582633
...,...,...,...
volatility_bbh,0.901071,-1.255339,-1.131150
volatility_kcw,0.804037,-1.467506,-1.179929
volatility_ui,0.467576,-2.684731,-1.255316
momentum_pvo_signal,0.809746,-1.736980,-1.406512
